In [7]:
# IMPORTAR LIBRERIAS
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
load_dotenv() #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify

import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)

In [8]:
#AUTENTICACION EN SPOTIFY PARA EXTRAER DATOS DE API

mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

In [3]:
# FUNCION PARA BUSCAR, EXTRAER Y CONVERTIR A DATAFRAME LA INFORMACION DE SPOTIFY

todas_las_canciones_df = None


def busqueda_spotify(generos, año):
    todas_las_canciones = []
    albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
    datos_artistas = {}
    datos_artistas = {} 
    artistas_procesados = set()

    print("Buscando canciones del año", año)

    for genero in generos:
        print("Género:", genero)

        # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
        #    No funciona con álbumes ni canciones directamente.
        resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
            #    al buscar álbumes (type="album") o canciones (type="track").
            #    Aquí buscamos álbumes del artista en el año indicado.
            busqueda = "artist:" + nombre_artista + " year:" + str(año)
            resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
            albumes = resultado_albumes["albums"]["items"]
            
            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]
                
                # Si ya vimos este álbum, saltamos al siguiente
                if id_album in albumes_ya_vistos:
                    continue
                
                # Si no lo habíamos visto, lo marcamos como visto
                albumes_ya_vistos.add(id_album)

                # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print("  → Canciones de este género añadidas")
        print()

    # Mostramos un resumen de lo encontrado
    print("RESUMEN:")

    # Contamos las canciones de cada género una por una
    for genero in generos:
        contador = 0
        for cancion in todas_las_canciones:
            if cancion["genero"] == genero:
                contador = contador + 1
        print(f"- {genero}: {contador} canciones")

    print("Total de canciones encontradas:", len(todas_las_canciones))

    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  
    return todas_las_canciones_df

In [4]:

genero = ["flamenco","latin","jazz","rock"]
año = 2014
todas_las_canciones_df = busqueda_spotify(genero,año)

Buscando canciones del año 2014
Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 6 canciones
- latin: 2806 canciones
- jazz: 5727 canciones
- rock: 3733 canciones
Total de canciones encontradas: 12272


In [5]:
# Mostrar contenido
todas_las_canciones_df 


,nombre,artista,album,genero,año
0,Life,LaRumbé,Sonido Obscuro,flamenco,2014
1,Boomerang,LaRumbé,Sonido Obscuro,flamenco,2014
2,Getaway Car,LaRumbé,Sonido Obscuro,flamenco,2014
3,Falling,LaRumbé,Sonido Obscuro,flamenco,2014
4,Dullness,LaRumbé,Sonido Obscuro,flamenco,2014
...,...,...,...,...,...
12267,By the Way,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
12268,Universally Speaking,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
12269,This Is the Place,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
12270,Dosed,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014


In [6]:
# PASAR DATAFRAME A UN ARCHIVO CSV
todas_las_canciones_df.to_csv("canciones_2014_ok.csv", index=False)


In [7]:
# AUTENTICACION EN LASTFM
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

In [8]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [9]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [10]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = todas_las_canciones_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)

    # 6. Unir los datos de Last.fm al DataFrame principal de canciones
    df_final = pd.merge(
        todas_las_canciones_df,
        datos_lastfm_df, # ¡Usamos el DF recién creado!
        on='artista',
        how='left'
    )
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 103

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [11]:
datos_lastfm_df

,artista,bio_resumen,listeners
0,LaRumbé,,256
1,Morat,Morat is a Colombian band formed in the countr...,292367
2,Don Omar,"William Omar Landrón (born February 10, 1978 i...",1703423
3,Duki,"Mauro Ezequiel Lombardo Quiroga, conocido artí...",283448
4,Milo j,Camilo Joaquín Villarruel nació el 25 de octub...,243123
...,...,...,...
98,Radiohead,Radiohead is an English rock band formed in Ab...,8011237
99,Queen,Queen are a British rock band formed in London...,7049351
100,Maná,There is more than one artist with this name:\...,817784
101,Maroon 5,Maroon 5 is an American pop rock band that ori...,6439053


In [12]:
datos_lastfm_df
datos_lastfm_df.to_csv("datos_lastfm_2014.csv", index=False)

In [10]:
import pandas as pd  # Asegúrate de importar pandas antes

todas_las_canciones_df = None

def busqueda_spotify(generos, año):
    todas_las_canciones = []

    print("Buscando canciones del año", año)

    for genero in generos:
        print("Género:", genero)

        # Buscamos canciones directamente usando type="track"
        busqueda = f'genre:"{genero}" year:{año}'
        resultado = spotify.search(q=busqueda, type="track", limit=50)
        canciones = resultado["tracks"]["items"]

        for cancion in canciones:
            info = {
                "nombre": cancion["name"],
                "artista": cancion["artists"][0]["name"],
                "album": cancion["album"]["name"],
                "genero": genero,
                "año": año
            }
            todas_las_canciones.append(info)

        print(f"  → {len(canciones)} canciones de este género añadidas\n")

    # Resumen por género
    print("RESUMEN:")
    for genero in generos:
        contador = sum(1 for c in todas_las_canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")

    print("Total de canciones encontradas:", len(todas_las_canciones))

    # Convertimos a DataFrame
    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)
    return todas_las_canciones_df


In [11]:
generos = ["flamenco","latin","jazz","rock"]
año = 2014
todas_las_canciones_df_1 = busqueda_spotify(generos,año)

Buscando canciones del año 2014
Género: flamenco
  → 50 canciones de este género añadidas

Género: latin
  → 50 canciones de este género añadidas

Género: jazz
  → 50 canciones de este género añadidas

Género: rock
  → 50 canciones de este género añadidas

RESUMEN:
- flamenco: 50 canciones
- latin: 50 canciones
- jazz: 50 canciones
- rock: 50 canciones
Total de canciones encontradas: 200
